In [ ]:
# Find Inner Point

In [ ]:
#| default_exp inner_point

In [1]:
#| export
from scipy.optimize import linprog
import numpy as np
import torch

from typing import Union
import gc

from mhar.polytope import Polytope, NFDPolytope

In [3]:
#| export

def ChebyshevCenter(
                    polytope:Union[Polytope, NFDPolytope], 
                    lb=None, # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None, # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance = 1e-4
                    ): 

    ## Equality          
    A_in = polytope.A_in.numpy()
    b_in = polytope.b_in.numpy()
    
    A_in_norm = np.matrix(np.sum(A_in ** 2., axis=-1) ** (1. / 2.))
    # Create new restriction matrices
    A_in_norm = np.concatenate((A_in, A_in_norm.transpose()), axis=1)
    c = np.concatenate((np.zeros(A_in.shape[1]), [-1.]))
    del A_in 
    gc.collect()
    
    ## Inequality
    if isinstance(polytope,NFDPolytope):
        mE = polytope.mE
        A_eq = polytope.A_eq
        # The equality restrictions have zero norm. Is transposed to keep order
        A_eq_norm = np.zeros((1, A_eq.shape[0]))
        # Create new restriction matrices
        A_eq_norm = np.concatenate((A_eq, A_eq_norm.transpose()), axis=1)
        del A_eq
        gc.collect()
        b_eq = polytope.b_eq
    else:
        mE=0
        A_eq_norm = None
        b_eq = None
        
    r = linprog(c=c,
        A_ub=A_in_norm,
        b_ub=b_in,
        A_eq=A_eq_norm,
        b_eq=b_eq,
        bounds = (lb, ub),
        method= 'revised simplex')

    status = {0:'Optimization proceeding nominally.',
                1: 'Iteration limit reached.',
                2: 'Problem appears to be infeasible.',
                3: ' Problem appears to be unbounded.',
                4: ' Numerical difficulties encountered.',
            }[r.status]
    
    print('\nSimplex Status for the Chebyshev Center\n', status)
    
    x0 = torch.tensor(np.array(r.x[:-1], ndmin=2).transpose(), polytope.dtype)
    b0 = torch.matmul(polytope.A_in, x0)
    
    assert(torch.all(b0 <= b_in )), f'Point {x0} does not satisfy A_inx <= b_in restrictions, it may have numerical inestability'
    if mE>0:
        assert(torch.all(torch.abs(b0 - b_eq) <= tolerance)), f'Point {x0} does not satisfy A_eqx = b_eq restrictions with tolerance {tolerance}, it may have numerical inestability'
    return x0
    
    

            
